# Strands Agent with Knowledge Base on AgentCore
Deploy a Strands agent that queries the equipment specs knowledge base to AgentCore

## 1. Load Knowledge Base Configuration

In [ ]:
import json
import boto3

# Load KB config from previous notebook
with open('kb_config.json', 'r') as f:
    kb_config = json.load(f)

KB_ID = kb_config['knowledge_base_id']
REGION = kb_config['region']

print(f"Knowledge Base ID: {KB_ID}")
print(f"Region: {REGION}")

## 2. Create Knowledge Base Tool for Strands

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel

bedrock_runtime = boto3.client('bedrock-agent-runtime', region_name=REGION)

@tool
def search_equipment_specs(query: str) -> str:
    """Search equipment specification documents for technical information.
    Use this for questions about equipment specs, capacities, dimensions, or features.
    """
    response = bedrock_runtime.retrieve_and_generate(
        input={'text': query},
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': KB_ID,
                'modelArn': f'arn:aws:bedrock:{REGION}::foundation-model/amazon.nova-pro-v1:0',
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'numberOfResults': 5
                    }
                }
            }
        }
    )
    return response['output']['text']

print("✓ Knowledge Base tool created")

## 3. Test Strands Agent Locally

In [ ]:
# Create Strands agent with KB tool
equipment_agent = Agent(
    system_prompt="You are an equipment specialist assistant. Use the search tool to answer questions about equipment specifications.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0", region=REGION),
    tools=[search_equipment_specs]
)

# Test locally
test_queries = [
    "What is the lifting capacity of the mobile crane?",
    "Compare the fuel capacities of all equipment",
    "What are the dimensions of the excavator?"
]

for query in test_queries:
    print(f"Q: {query}")
    response = equipment_agent(query)
    print(f"A: {response}\n")
    print("-" * 80 + "\n")

## 4. Create AgentCore Deployment File

In [ ]:
agent_code = f'''
import boto3
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

KB_ID = "{KB_ID}"
REGION = "{REGION}"

bedrock_runtime = boto3.client('bedrock-agent-runtime', region_name=REGION)

@tool
def search_equipment_specs(query: str) -> str:
    """Search equipment specification documents for technical information.
    Use this for questions about equipment specs, capacities, dimensions, or features.
    """
    response = bedrock_runtime.retrieve_and_generate(
        input={{'text': query}},
        retrieveAndGenerateConfiguration={{
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {{
                'knowledgeBaseId': KB_ID,
                'modelArn': f'arn:aws:bedrock:{{REGION}}::foundation-model/amazon.nova-pro-v1:0',
                'retrievalConfiguration': {{
                    'vectorSearchConfiguration': {{
                        'numberOfResults': 5
                    }}
                }}
            }}
        }}
    )
    return response['output']['text']

agent = Agent(
    system_prompt="You are an equipment specialist assistant. Use the search tool to answer questions about equipment specifications.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0", region=REGION),
    tools=[search_equipment_specs]
)

@app.entrypoint
def invoke(payload):
    prompt = payload.get("prompt", "Hello!")
    response = agent(prompt)
    return {{"result": str(response)}}

if __name__ == "__main__":
    app.run()
'''

with open('equipment_agent.py', 'w') as f:
    f.write(agent_code)

print("✓ Agent saved to equipment_agent.py")

## 5. Create Requirements File

In [ ]:
requirements = '''bedrock-agentcore
strands-agents
boto3
'''

with open('requirements.txt', 'w') as f:
    f.write(requirements)

print("✓ Requirements saved to requirements.txt")

## 6. Test Agent Locally (Optional)

In [ ]:
# Test the deployment file locally
import subprocess
import time
import requests

# Start agent in background (uncomment to test)
# process = subprocess.Popen(['python', 'equipment_agent.py'])
# time.sleep(5)

# Test with curl
# response = requests.post(
#     'http://localhost:8080/invocations',
#     json={'prompt': 'What is the lifting capacity of the mobile crane?'}
# )
# print(response.json())

# Stop agent
# process.terminate()

print("Local testing skipped - deploy to AgentCore instead")

## 7. Deploy to AgentCore

Run these commands in terminal:

```bash
# Configure agent
agentcore configure -e equipment_agent.py -r us-east-1

# Deploy to AgentCore
agentcore deploy

# Note the agent ARN from output
```

## 8. Test Deployed Agent

In [ ]:
# After deployment, test with agentcore CLI
# agentcore invoke '{"prompt": "What equipment do we have?"}'

print("Use agentcore CLI to test deployed agent")

## 9. Invoke Deployed Agent Programmatically

In [ ]:
import uuid

def invoke_equipment_agent(agent_arn: str, prompt: str):
    """Invoke deployed equipment agent."""
    client = boto3.client('bedrock-agentcore', region_name=REGION)
    
    payload = json.dumps({"prompt": prompt}).encode()
    
    response = client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        runtimeSessionId=str(uuid.uuid4()),
        payload=payload,
        qualifier="DEFAULT"
    )
    
    content = []
    for chunk in response.get("response", []):
        content.append(chunk.decode('utf-8'))
    
    return json.loads(''.join(content))

# Example usage (replace with your agent ARN after deployment)
# AGENT_ARN = "arn:aws:bedrock-agentcore:us-east-1:123456789012:agent-runtime/equipment-agent"
# result = invoke_equipment_agent(AGENT_ARN, "What is the fuel capacity of the excavator?")
# print(result['result'])

print("Function ready - add your agent ARN after deployment")

## 10. Create Multi-Agent Version (Advanced)

In [ ]:
multi_agent_code = f'''
import boto3
from strands import Agent, tool
from strands.models import BedrockModel
from strands.multiagent import Swarm
from bedrock_agentcore import BedrockAgentCoreApp

app = BedrockAgentCoreApp()

KB_ID = "{KB_ID}"
REGION = "{REGION}"

bedrock_runtime = boto3.client('bedrock-agent-runtime', region_name=REGION)

@tool
def search_equipment_specs(query: str) -> str:
    """Search equipment specifications."""
    response = bedrock_runtime.retrieve_and_generate(
        input={{'text': query}},
        retrieveAndGenerateConfiguration={{
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {{
                'knowledgeBaseId': KB_ID,
                'modelArn': f'arn:aws:bedrock:{{REGION}}::foundation-model/amazon.nova-pro-v1:0',
                'retrievalConfiguration': {{
                    'vectorSearchConfiguration': {{'numberOfResults': 5}}
                }}
            }}
        }}
    )
    return response['output']['text']

# Create specialized agents
researcher = Agent(
    name="researcher",
    system_prompt="Research equipment specs and hand off to analyst.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0", region=REGION),
    tools=[search_equipment_specs]
)

analyst = Agent(
    name="analyst",
    system_prompt="Analyze equipment data and hand off to writer.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0", region=REGION)
)

writer = Agent(
    name="writer",
    system_prompt="Create clear equipment summaries.",
    model=BedrockModel(model_id="us.amazon.nova-lite-v1:0", region=REGION)
)

swarm = Swarm(
    agents=[researcher, analyst, writer],
    max_handoffs=2,
    max_iterations=2,
    execution_timeout=180.0
)

@app.entrypoint
def invoke(payload):
    prompt = payload.get("prompt", "Hello!")
    result = swarm(prompt)
    return {{
        "result": str(result.final_response),
        "path": [node.node_id for node in result.node_history]
    }}

if __name__ == "__main__":
    app.run()
'''

with open('equipment_swarm_agent.py', 'w') as f:
    f.write(multi_agent_code)

print("✓ Multi-agent version saved to equipment_swarm_agent.py")

## Summary

### Created Files:

1. **equipment_agent.py** - Single agent with KB integration
2. **equipment_swarm_agent.py** - Multi-agent swarm version
3. **requirements.txt** - Dependencies

### Architecture:

```
User → AgentCore Runtime → Strands Agent → KB Tool → Bedrock KB → S3 Vectors
```

### Deployment Commands:

```bash
# Single agent
agentcore configure -e equipment_agent.py -r us-east-1
agentcore deploy
agentcore invoke '{"prompt": "What equipment do we have?"}'

# Multi-agent swarm
agentcore configure -e equipment_swarm_agent.py -r us-east-1
agentcore deploy
agentcore invoke '{"prompt": "Compare all equipment capacities"}'
```

### Benefits:

- **Strands**: Simple, flexible agent development
- **Knowledge Base**: RAG with equipment specs
- **S3 Vectors**: 90% cost savings
- **AgentCore**: Production-ready serverless deployment

### Next Steps:

1. Deploy to AgentCore
2. Test with various equipment queries
3. Monitor with CloudWatch
4. Add more tools (calculations, comparisons)
5. Integrate with applications